In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# Define the UNet model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()  # Apply sigmoid to output probabilities
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [3]:
# Dataset class to load images and masks
class GlacierDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))
        self.mask_files = sorted(os.listdir(mask_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        # Ensure mask is 2D (single channel) after transformations
        if len(mask.shape) == 3:
            mask = mask[:, :, 0]  # If mask has multiple channels, take one channel
        
        return image, mask


In [4]:
# Padding function to make the image dimensions a multiple of 32
def pad_to_nearest_32(image):
    h, w = image.shape[:2]
    pad_h = (32 - h % 32) % 32
    pad_w = (32 - w % 32) % 32
    padded_image = cv2.copyMakeBorder(image, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT, value=0)
    return padded_image

In [5]:
# Train model function
def train_model(model, train_loader, device, num_epochs, lr):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device).float()
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            
            # Loss calculation
            loss = criterion(outputs, masks)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [6]:
# Prediction Function
def predict_mask(image_path, model, device):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = T.Compose([T.ToTensor()])
    image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        mask = torch.sigmoid(output).squeeze(0).cpu().numpy()
        mask = (mask > 0.5).astype(np.uint8)
    return mask[0]  # Assuming binary segmentation

In [7]:
# Post-processing techniques
def morphological_operations(mask):
    kernel = np.ones((5, 5), np.uint8)
    mask_closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask_opened = cv2.morphologyEx(mask_closed, cv2.MORPH_OPEN, kernel)
    return mask_opened

In [8]:
# Noise reduction using Gaussian blur
def reduce_noise(mask):
    return cv2.GaussianBlur(mask, (5, 5), 0)


In [9]:
# Edge refinement using Canny edge detection
def refine_boundaries(mask):
    return cv2.Canny(mask, threshold1=100, threshold2=200)

In [10]:
def post_process_mask(mask):
    # Ensure the mask is 2D grayscale before processing
    if len(mask.shape) != 2:
        raise ValueError("The mask should be a 2D image")
    
    mask = morphological_operations(mask)
    mask = reduce_noise(mask)
    mask = refine_boundaries(mask)
    return mask

In [11]:
# Difference Mapping
def calculate_difference(before_mask, after_mask):
    # Calculate the difference between the before and after masks
    difference_mask = np.abs(before_mask - after_mask)
    
    # Optional: Use morphological operations to clean the difference map
    difference_mask = morphological_operations(difference_mask)
    
    return difference_mask

In [12]:
# Area Calculation
def calculate_area(mask):
    # Calculate the area of the segmented region
    return np.sum(mask)  # This will count the number of non-zero pixels (i.e., segmented area)


In [13]:
# Visualizing Results
# Visualize overlays
def overlay_mask(image_path, mask, color="red"):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    # Resize mask to match image dimensions
    mask_resized = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)
    
    # Create a colored mask
    mask_colored = np.zeros((h, w, 3), dtype=np.uint8)
    
    if color == "red":
        mask_colored[..., 2] = mask_resized * 255  # Red channel
    elif color == "green":
        mask_colored[..., 1] = mask_resized * 255  # Green channel
    elif color == "blue":
        mask_colored[..., 0] = mask_resized * 255  # Blue channel

    # Overlay the mask on the image
    overlayed_image = cv2.addWeighted(image, 1.0, mask_colored, 0.5, 0)
    
    return overlayed_image


In [14]:
# Visualization function
def visualize_and_calculate(before_image_path, after_image_path, model, device):
    before_mask = predict_mask(before_image_path, model, device)
    after_mask = predict_mask(after_image_path, model, device)

    before_mask = post_process_mask(before_mask)
    after_mask = post_process_mask(after_mask)

    difference_mask = calculate_difference(before_mask, after_mask)

    before_area = calculate_area(before_mask)
    after_area = calculate_area(after_mask)
    difference_area = calculate_area(difference_mask)

    print(f"Area of before mask: {before_area} pixels")
    print(f"Area of after mask: {after_area} pixels")
    print(f"Area of difference: {difference_area} pixels")

    before_overlay = overlay_mask(before_image_path, before_mask, color="red")
    after_overlay = overlay_mask(after_image_path, after_mask, color="red")
    difference_overlay = overlay_mask(before_image_path, difference_mask, color="green")

    plt.figure(figsize=(20, 10))
    plt.subplot(1, 3, 1)
    plt.title("Before")
    plt.imshow(cv2.cvtColor(before_overlay, cv2.COLOR_BGR2RGB))
    plt.subplot(1, 3, 2)
    plt.title("After")
    plt.imshow(cv2.cvtColor(after_overlay, cv2.COLOR_BGR2RGB))
    plt.subplot(1, 3, 3)
    plt.title("Difference")
    plt.imshow(cv2.cvtColor(difference_overlay, cv2.COLOR_BGR2RGB))
    plt.show()

In [15]:
# Main function
if __name__ == "__main__":
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    train_image_dir = "data/images"
    train_mask_dir = "data/masks"
    before_image_path = "before.png"
    after_image_path = "after.png"

    transform = T.Compose([T.ToTensor()])
    train_dataset = GlacierDataset(train_image_dir, train_mask_dir, transform)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

    model = UNet(in_channels=3, out_channels=1).to(device)

    train_model(model, train_loader, device, num_epochs=10, lr=0.001)

    visualize_and_calculate(before_image_path, after_image_path, model, device)

ValueError: Using a target size (torch.Size([4, 1, 400])) that is different to the input size (torch.Size([4, 1, 400, 400])) is deprecated. Please ensure they have the same size.